In [1]:
import torch
mat = torch.Tensor([[1,2,3],[4,5,6],[7,8,9]])
print(mat)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [2]:
mask = torch.LongTensor([1,0,1]).reshape(3,1)
print(mask.size())
print(mat*mask)
print(torch.mul(mat, mask))
print(torch.mul(mat, mask.reshape(1,3)))

torch.Size([3, 1])
tensor([[1., 2., 3.],
        [0., 0., 0.],
        [7., 8., 9.]])
tensor([[1., 2., 3.],
        [0., 0., 0.],
        [7., 8., 9.]])
tensor([[1., 0., 3.],
        [4., 0., 6.],
        [7., 0., 9.]])


In [3]:
print(torch.mm(mat, mask.float()))

tensor([[ 4.],
        [10.],
        [16.]])


In [5]:
print(mat.to_sparse())
print(mat.to_sparse().t())

tensor(indices=tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
                       [0, 1, 2, 0, 1, 2, 0, 1, 2]]),
       values=tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.]),
       size=(3, 3), nnz=9, layout=torch.sparse_coo)
tensor(indices=tensor([[0, 1, 2, 0, 1, 2, 0, 1, 2],
                       [0, 0, 0, 1, 1, 1, 2, 2, 2]]),
       values=tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.]),
       size=(3, 3), nnz=9, layout=torch.sparse_coo)


In [3]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))

2022-03-25-16-38-52


In [5]:
import torch
from torch_sparse import SparseTensor

edge_index = torch.LongTensor([[1,0,0], [2,1,1]])
num_nodes = 3
adj = SparseTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(num_nodes, num_nodes))

In [11]:
print(adj)

SparseTensor(row=tensor([0, 0, 1]),
             col=tensor([1, 1, 2]),
             size=(3, 3), nnz=3, density=33.33%)


In [13]:
from torch_geometric.nn import GCNConv, GINConv  #, Sequential
from torch.nn import ReLU, Linear, Sequential

x = torch.FloatTensor([[1,1,1], [0.5,0.5,0.5], [2,2,2]])
conv = GCNConv(3, 32)
out1 = conv(x, edge_index)
out2 = conv(x, adj.t())
assert torch.allclose(out1, out2)

conv = GINConv(nn=Sequential(Linear(3, 32), ReLU(), Linear(32, 32)))
out1 = conv(x, edge_index)
out2 = conv(x, adj.t())
assert torch.allclose(out1, out2)

In [14]:
print(edge_index)
print(adj.t())

tensor([[1, 0, 0],
        [2, 1, 1]])
SparseTensor(row=tensor([1, 1, 2]),
             col=tensor([0, 0, 1]),
             size=(3, 3), nnz=3, density=33.33%)


In [15]:
print(adj.t().size(0))

3


In [19]:
print(type(edge_index.to_sparse()))

<class 'torch.Tensor'>


In [20]:
sparse_edge_index = edge_index.to_sparse()
sparse_edge_index = SparseTensor(sparse_edge_index)
print(sparse_edge_index)

AssertionError: 

In [1]:
import datetime
time_str = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
time_str

'2022-03-26-11-50-22'